In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [2]:
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse

p = Path('.')

json_string='''@article{asness2000predicting,
  title={Predicting stock returns using industry-relative firm characteristics},
  author={Asness, Clifford S and Porter, R Burt and Stevens, Ross L},
  journal={Available at SSRN 213872},
  year={2000}
}'''

url= f'https://scholar.google.com/scholar?cites=2542132931406996657&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input

collect_dict_values= {}
for item in json_string.split('\n'):
    ii= item.split('=')
    if 'year' in [i.strip() for i in ii ]:
        collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
    if 'title' in [i.strip() for i in ii ]:
        collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
    if 'author' in [i.strip() for i in ii ]:
        collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
        # print(re.sub('\W+','-', item.split('=')[1]))
    if 'journal' in [i.strip() for i in ii ]:
        collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
        
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]

print('CITATION KEY: ', citation_key)

citation_key =f'{citation_key}' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')

citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')

!clear
!git status
!export citation_key=($citation_key)
!git add .
!git commit -m "$citation_key"
!git push

citation_dict

article--2000--Lee-Charles-MC-and-Swaminathan-Bhaskaran--Price-momentum-and-trading-volume--the-Journal-of-Finance.pdf
CITATION KEY:  lee2000price
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   academic_citation_network.csv
	modified:   academic_citation_networks_google_scholar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main 9b2a80a] lee2000price
 2 files changed, 12 insertions(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 16 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.17 KiB | 399.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/krishpn/academic_graphs.git

{'citation_key': 'lee2000price',
 'google_scholar_link': 'https://scholar.google.com/scholar?start=20&hl=en&as_sdt=5,47&sciodt=0,47&cites=4023483204291451485&scipsc=',
 'cited_by': [{'article_title': 'Experimental research in financial accounting',
   'article_link': 'https://www.sciencedirect.com/science/article/pii/S0361368201000113',
   'cited_by': '/scholar?cites=9002085329117032196&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '1586',
   'auhtors': ['R Libby', 'R Bloomfield', 'MW Nelson']},
  {'article_title': 'Game on: Social networks and markets',
   'article_link': 'https://www.sciencedirect.com/science/article/pii/S0304405X22000964',
   'cited_by': '/scholar?cites=13292618557708041496&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '154',
   'auhtors': ['LH Pedersen']},
  {'article_title': 'Market reactions to tangible and intangible information',
   'article_link': 'https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1540-6261.2006.00884.x',
   'cited_by': '/schol